In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
import nbformat
print(nbformat.__version__)


5.10.4


In [4]:
df = pd.read_csv('/Users/mainoahmuna/Google Drive/My Drive/Team-Fermata-Energy/processed_data/processed_weather_and_load/32.csv')

In [5]:
PATH = '/Users/mainoahmuna/Google Drive/My Drive/Team-Fermata-Energy/processed_data/processed_weather_and_load/'

In [6]:
df.head()

,Index,out.electricity.total.energy_consumption,Dry Bulb Temperature [°C],Relative Humidity [%],heat_index,hour,month,is_weekday,is_holiday,max_load_hourly,max_temp_hourly,min_temp_hourly,bldg_id
0,0,5.420033,-6.10,42.781847,21.02,1,1,1,0,5.476113,-6.1,-6.55,32
1,1,5.476113,-6.25,43.350762,20.75,1,1,1,0,5.476113,-6.1,-6.55,32
2,2,5.476113,-6.40,43.919676,20.48,1,1,1,0,5.476113,-6.1,-6.55,32
3,3,5.476113,-6.55,44.488591,20.21,1,1,1,0,5.476113,-6.1,-6.55,32
4,4,5.476113,-6.70,45.057505,19.94,2,1,1,0,5.476113,-6.7,-6.70,32


In [7]:
df.shape

(35037, 13)

In [8]:
buildings = df['bldg_id'].tolist()

In [9]:
def smape(y_true, y_pred):
    """
    Calculate Symmetric Mean Absolute Percentage Error (SMAPE).

    Parameters:
        y_true (array-like): True values.
        y_pred (array-like): Predicted values.

    Returns:
        float: SMAPE value.
    """
    numerator = np.abs(y_true - y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    smape_value = np.mean(numerator / denominator) * 100
    return smape_value

In [10]:
def create_Y_X(df_load):
    """
    Create Y and X variables for linear regression model.

    Parameters:
        df_load (pandas.DataFrame): DataFrame containing load data.

    Returns:
        tuple: Tuple containing Y and X variables.  
    """
    Y = df_load['out.electricity.total.energy_consumption']
    X = df_load.drop('out.electricity.total.energy_consumption', axis=1)
    return Y, X

In [69]:
def plot_actual_vs_predicted(actual, predicted):
    """
    Plot actual vs predicted values using Plotly.

    Parameters:
        actual (pandas.Series): Actual values.
        predicted (numpy.ndarray): Predicted values.

    Returns:
        fig: Plotly figure object.
    """
    fig = go.Figure()

    # Add actual values
    fig.add_trace(go.Scatter(x=actual.index, y=actual, mode='lines', name='Actual', line=dict(color='blue')))

    # Add predicted values using the same index as the actual values
    fig.add_trace(go.Scatter(x=actual.index, y=predicted, mode='lines', name='Predicted', line=dict(color='red')))

    # Update the layout
    fig.update_layout(
        title="Actual vs Predicted Values",
        xaxis_title="Index",
        yaxis_title="Energy Consumption",
        template="plotly_white"
    )
    
    fig.show()
    return fig

In [75]:
def train_sgd_regressor(directory, target_column='out.electricity.total.energy_consumption', test_size=0.2, chunk_size=1000):
    """
    Train the SGD Regressor model using data from a chunk of building CSV files.

    Parameters:
        directory (str): Path to the directory containing building CSV files.
        target_column (str): The name of the target column.
        test_size (float): Proportion of the dataset to include in the test split.
        chunk_size (int): Number of rows to process in each chunk.

    Returns:
        model: Trained model.
        building_pred: Predictions for one building.
        building_actual: Actual values for that building.
        avg_smape: Average SMAPE across all buildings.
        avg_r2: Average R² score across all buildings.
    """
    model = SGDRegressor()

    # Lists to store individual SMAPE and R² scores
    smape_list = []
    building_pred = None
    building_actual = None

    # Use glob to list all CSV files in the directory
    csv_files = glob(f"{directory}/*.csv")
    

    # Training phase: loop over the selected building files
    for filename in tqdm(csv_files, desc="Processing buildings", unit="file"):
        try:
            # Use chunksize in read_csv to read the file in chunks
            for chunk in pd.read_csv(filename, chunksize=chunk_size):
                Y, X = create_Y_X(chunk)

                # Train the model with the current chunk
                model.partial_fit(X, Y)

        except Exception as e:
            print(f"Error processing file {filename}: {e}")
            continue  # Skip to the next file if there's an error

    # Prepare to evaluate the model using the last file processed for testing
    test_file = selected_files[-1]  # Use the last file for testing (or choose a specific file)
    test_data = pd.read_csv(test_file)
    Y_test, X_test = create_Y_X(test_data)

    # Make predictions on the test set
    predictions = model.predict(X_test)

    # Calculate SMAPE and R²
    smape_value = smape(Y_test, predictions)

    # Store predictions for one building (the last one processed)
    building_pred = predictions
    building_actual = Y_test

    print('Average Metrics:')
    print(f'Average SMAPE: {smape_value:.2f}%')

    return model, building_pred, building_actual, smape_value,

In [76]:
model, building_pred, building_actual, avg_smape = train_linear_regression(PATH)

Processing buildings:   9%|▉         | 557/6000 [03:20<32:40,  2.78file/s]  


KeyboardInterrupt: 

## Linear Regression:
Average SMAPE: 11.74%


In [78]:
building_pred = pd.DataFrame(building_pred)

In [79]:
building_pred.head()

,0
0,0.526859
1,4.413343
2,7.895777
3,17.388664
4,2.843109


In [74]:
plot_actual_vs_predicted(building_actual, building_pred)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed